In [45]:
import awkward as ak
import os
import subprocess
import shutil
import glob

In [61]:
region = 'singlemuon'
dset = 'ttboosted_1000toInf_part_'

In [62]:
for i in os.listdir(f'/project01/ndcms/cmoore24/skims/full_skims/{region}/mc/2017'):
    if (f'{dset}' in i):
        try:
            print(f'Processing {i}')
            arr = ak.from_parquet(f'/project01/ndcms/cmoore24/skims/full_skims/{region}/mc/2017/{i}/part*')
            size = len(arr)
            diff = size / 2
            for j in range(0,2):
                if j < 1:
                    ak.to_parquet(arr[diff*j:diff*(j+1)], f'/project01/ndcms/cmoore24/skims/full_skims/{region}/mc/2017/{i}/keep{j}.parquet')
                else:
                    ak.to_parquet(arr[diff*j:], f'/project01/ndcms/cmoore24/skims/full_skims/{region}/mc/2017/{i}/keep{j}.parquet')
    
            subprocess.run(f'find /project01/ndcms/cmoore24/skims/full_skims/{region}/mc/2017/{i} -name "part*.parquet" -exec rm {{}} +', shell=True, check=True)
        except:
            continue

Processing ttboosted_1000toInf_part_001
Processing ttboosted_1000toInf_part_002
Processing ttboosted_1000toInf_part_003
Processing ttboosted_1000toInf_part_004
Processing ttboosted_1000toInf_part_005
Processing ttboosted_1000toInf_part_006


In [63]:
base_path = f'/project01/ndcms/cmoore24/skims/full_skims/{region}/mc/2017'
output_dirs = {}

counter = 0

for d in sorted(os.listdir(base_path)):
    full_dir = os.path.join(base_path, d)
    if not os.path.isdir(full_dir):
        continue
    if not d.startswith(f'{dset}'):
        continue

    base_name = d.rsplit('_', 2)[0]
    out_dir = os.path.join(base_path, base_name)
    os.makedirs(out_dir, exist_ok=True)

    for f in sorted(glob.glob(os.path.join(full_dir, 'keep*.parquet'))):
        new_name = f'keep{counter}.parquet'
        new_path = os.path.join(out_dir, new_name)
        shutil.move(f, new_path)
        counter += 1

    shutil.rmtree(full_dir)
